<a href="https://colab.research.google.com/github/SanjayNSanju/CTC-Prediction/blob/main/CTC_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Importing Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [154]:
import numpy as np
import pandas as pd
#import xgboost as xgb
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

# Loading Datasets.

In [23]:
train = pd.read_csv("/content/drive/MyDrive/My_Projects/CTC Prediction/Train.csv")

test = pd.read_csv("/content/drive/MyDrive/My_Projects/CTC Prediction/Train.csv")

# Processing Datasets.

In [24]:
train.head()

,S.No.,College,Role,City type,Previous CTC,Previous job changes,Graduation marks,Exp,CTC
0,1,Tier 1,Manager,Non-Metro,"55,523.00",3,66,19,"71,406.58"
1,2,Tier 2,Executive,Metro,"57,081.00",1,84,18,"68,005.87"
2,3,Tier 2,Executive,Metro,"60,347.00",2,52,28,"76,764.02"
3,4,Tier 3,Executive,Metro,"49,010.00",2,81,33,"82,092.39"
4,5,Tier 3,Executive,Metro,"57,879.00",4,74,32,"73,878.10"


In [25]:
test.head()

,S.No.,College,Role,City type,Previous CTC,Previous job changes,Graduation marks,Exp,CTC
0,1,Tier 1,Manager,Non-Metro,"55,523.00",3,66,19,"71,406.58"
1,2,Tier 2,Executive,Metro,"57,081.00",1,84,18,"68,005.87"
2,3,Tier 2,Executive,Metro,"60,347.00",2,52,28,"76,764.02"
3,4,Tier 3,Executive,Metro,"49,010.00",2,81,33,"82,092.39"
4,5,Tier 3,Executive,Metro,"57,879.00",4,74,32,"73,878.10"


In [26]:
del train['S.No.']
del test['S.No.']

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   College               1338 non-null   object
 1   Role                  1338 non-null   object
 2   City type             1338 non-null   object
 3   Previous CTC          1338 non-null   object
 4   Previous job changes  1338 non-null   int64 
 5   Graduation marks      1338 non-null   int64 
 6   Exp                   1338 non-null   int64 
 7   CTC                   1338 non-null   object
dtypes: int64(3), object(5)
memory usage: 83.8+ KB


In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   College               1338 non-null   object
 1   Role                  1338 non-null   object
 2   City type             1338 non-null   object
 3   Previous CTC          1338 non-null   object
 4   Previous job changes  1338 non-null   int64 
 5   Graduation marks      1338 non-null   int64 
 6   Exp                   1338 non-null   int64 
 7   CTC                   1338 non-null   object
dtypes: int64(3), object(5)
memory usage: 83.8+ KB


In [29]:
train=train.replace({',': ''}, regex=True)

test=test.replace({',': ''}, regex=True)

# Converting Objects to int and float .

In [30]:
train['CTC'] = train['CTC'].astype(np.float64)

train['Previous CTC']=train['Previous CTC'].astype(np.float64)

test['CTC']=test['CTC'].astype(np.float64)

test['Previous CTC']=test['Previous CTC'].astype(np.float64)

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   College               1338 non-null   object 
 1   Role                  1338 non-null   object 
 2   City type             1338 non-null   object 
 3   Previous CTC          1338 non-null   float64
 4   Previous job changes  1338 non-null   int64  
 5   Graduation marks      1338 non-null   int64  
 6   Exp                   1338 non-null   int64  
 7   CTC                   1338 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 83.8+ KB


In [32]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   College               1338 non-null   object 
 1   Role                  1338 non-null   object 
 2   City type             1338 non-null   object 
 3   Previous CTC          1338 non-null   float64
 4   Previous job changes  1338 non-null   int64  
 5   Graduation marks      1338 non-null   int64  
 6   Exp                   1338 non-null   int64  
 7   CTC                   1338 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 83.8+ KB


# Label Encoding.

In [33]:
col = ['College','Role','City type']

for c in col:
    le = LabelEncoder()
    train[c] = le.fit_transform(train[c])
    test[c] = le.fit_transform(test[c])

In [34]:
train.describe()

,College,Role,City type,Previous CTC,Previous job changes,Graduation marks,Exp,CTC
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,0.757848,0.204783,0.494768,55581.762332,2.525411,59.890882,39.207025,75459.339036
std,0.818422,0.403694,0.500160,6685.600553,1.123502,14.894696,14.049960,12551.280147
min,0.000000,0.000000,0.000000,36990.000000,1.000000,35.000000,18.000000,53020.320000
25%,0.000000,0.000000,0.000000,50547.000000,2.000000,47.000000,27.000000,66905.067500
50%,1.000000,0.000000,0.000000,55293.500000,3.000000,60.000000,39.000000,73112.640000
75%,1.000000,0.000000,1.000000,60150.500000,4.000000,73.000000,51.000000,80747.175000
max,2.000000,1.000000,1.000000,77911.000000,4.000000,85.000000,64.000000,123416.990000


In [35]:
train.corr()

,College,Role,City type,Previous CTC,Previous job changes,Graduation marks,Exp,CTC
College,1.000000,-0.001472,0.002384,-0.016259,-0.050243,-0.024994,-0.006760,-0.179152
Role,-0.001472,1.000000,-0.076185,0.021208,-0.013131,0.014541,-0.025019,0.625435
City type,0.002384,-0.076185,1.000000,-0.036956,0.001568,-0.018550,0.020856,-0.211258
Previous CTC,-0.016259,0.021208,-0.036956,1.000000,0.008282,-0.037170,0.117035,0.270260
Previous job changes,-0.050243,-0.013131,0.001568,0.008282,1.000000,0.001507,0.034137,0.007518
Graduation marks,-0.024994,0.014541,-0.018550,-0.037170,0.001507,1.000000,-0.065412,-0.017557
Exp,-0.006760,-0.025019,0.020856,0.117035,0.034137,-0.065412,1.000000,0.301569
CTC,-0.179152,0.625435,-0.211258,0.270260,0.007518,-0.017557,0.301569,1.000000


In [36]:
train.head()

,College,Role,City type,Previous CTC,Previous job changes,Graduation marks,Exp,CTC
0,0,1,1,55523.0,3,66,19,71406.58
1,1,0,0,57081.0,1,84,18,68005.87
2,1,0,0,60347.0,2,52,28,76764.02
3,2,0,0,49010.0,2,81,33,82092.39
4,2,0,0,57879.0,4,74,32,73878.10


In [37]:
test.head()

,College,Role,City type,Previous CTC,Previous job changes,Graduation marks,Exp,CTC
0,0,1,1,55523.0,3,66,19,71406.58
1,1,0,0,57081.0,1,84,18,68005.87
2,1,0,0,60347.0,2,52,28,76764.02
3,2,0,0,49010.0,2,81,33,82092.39
4,2,0,0,57879.0,4,74,32,73878.10


# Splitting Dataset into Training and Testing Set.

In [38]:
o_train = train['CTC']
del train['CTC']
i_train = train

o_test = test['CTC']
del test['CTC']
i_test = test

# Model Building and Training.

In [155]:
model = RandomForestRegressor(max_depth=20, random_state=0)

In [156]:
model.fit(i_train,o_train)

RandomForestRegressor(max_depth=20, random_state=0)

# Testing the Model.

In [157]:
pred = model.predict(i_test)

# Measuring Accuracy of the Model.

In [158]:
metrics.r2_score(o_test, pred)

0.9472913562527894

In [170]:
import warnings
warnings.filterwarnings('ignore')

In [171]:
print(model.predict([[1,0,0,57000,1,84,18]]))

[66244.5173]
